In [9]:
import sympy as sp
import symbtools as st


In [10]:
x1, x2, x3= xx = sp.symbols('x1:4') 
u = sp.Symbol("u_pwm")
print(xx,u)

(x1, x2, x3) u_pwm


In [11]:
#A_B,A_SP,m,g,T_M,k_M,k_V,k_L,eta_0,h_ub, h_lb,eta_ub,eta_lb,T,N_pred,state_r,input_r,h,h_p,eta,u_pwm = sp.symbols('A_B,A_SP,m,g,T_M,k_M,k_V,k_L,eta_0,h_ub, h_lb,eta_ub,eta_lb,T,N_pred,state_r,input_r,h,h_p,eta,u_pwm')
A_B= 2.8274e-3  # [m**2]
A_SP = 0.4299e-3  # [m**2]
m = 2.8e-3  # [kg]
g = 9.81  # [m/(s**2)]
T_M = 0.57  # [s]
k_M = 0.31  # [s**-1]
k_V = 6e-5  # [m**3]
k_L = 2.18e-4  # [kg/m]
eta_0 = 1900 / 60  # / 60 * 2 * pi
h_r,h_pr,eta_r=sp.symbols('h_r,h_pr,eta_r')
h_r=0.8

In [12]:
f_sys = sp.Matrix([x2, k_L / m * ((k_V * (x3 + eta_0) - A_B * x2) / A_SP)**2 - g, -1 / T_M * x3 ])
print(f)

Matrix([[x2], [0.0778571428571429*(-6.57687834380088*x2 + 0.139567341242149*x3 + 4.41963247266806)**2 - 9.81], [-1.75438596491228*x3]])


In [13]:
g_sys = sp.Matrix([0, 0,  k_M / T_M * u])
print(g_sys)

9.81


In [14]:
y=h=x1

In [15]:
Lfh = st.lie_deriv(h, f_sys, xx) ##:
Lf2h = st.lie_deriv(h, f_sys, xx, 2) ##:
Lf3h = st.lie_deriv(h, f_sys, xx, 3) ##:
print(Lf3h)

-1.75438596491228*x3*(-0.142932856041132*x2 + 0.00303316522687554*x3 + 0.0960502321843922) + (0.0778571428571429*(-6.57687834380088*x2 + 0.139567341242149*x3 + 4.41963247266806)**2 - 9.81)*(6.7354726195116*x2 - 0.142932856041132*x3 - 4.52620710796917)


In [16]:
Lgh = st.lie_deriv(h, g_sys, xx) ##:
LgLfh = st.lie_deriv(Lfh, g_sys, xx) ##:
Lf2h = st.lie_deriv(h, f_sys, xx, 2) ##:
LgLf2h=st.lie_deriv(Lf2h, g_sys, xx) 
print("LgLf2h",LgLf2h)

LgLf2h 0.543859649122807*u_pwm*(-0.142932856041132*x2 + 0.00303316522687554*x3 + 0.0960502321843922)


Relative Grad= 3

Bynis-Isidori-Normalform

In [17]:
T=sp.Matrix([h,Lfh,Lf2h])
print(T)

Matrix([[x1], [x2], [0.0778571428571429*(-6.57687834380088*x2 + 0.139567341242149*x3 + 4.41963247266806)**2 - 9.81]])


In [18]:
zz = st.symb_vector("z1:4")

res = sp.solve(T-zz, xx)[0]
x_z_beziehung = st.lzip(xx, res)
print("res",res)
print("x_z_beziehung",x_z_beziehung)

res (z1, z2, 47.1233333333334*z2 - 2.56783411036396*sqrt(100.0*z3 + 981.0) - 31.6666666666667)
x_z_beziehung [(x1, z1), (x2, z2), (x3, 47.1233333333334*z2 - 2.56783411036396*sqrt(100.0*z3 + 981.0) - 31.6666666666667)]


In [19]:
z_dot_tmp = T.jacobian(xx)*(f_sys + g_sys*u) 

z_dot = z_dot_tmp.subs(x_z_beziehung)
z_dot.simplify()
z_dot
print(z_dot)

Matrix([[z2], [0.0778571428571429*(-1.24344978758018e-14*z2 + 0.358385779534397*sqrt(100.0*z3 + 981.0) + 7.99360577730113e-15)**2 - 9.81], [(0.0778571428571429*(-1.24344978758018e-14*z2 + 0.358385779534397*sqrt(100.0*z3 + 981.0) + 7.99360577730113e-15)**2 - 9.81)*(-1.33226762955019e-14*z2 + 0.367027863234159*sqrt(100.0*z3 + 981.0) + 7.99360577730113e-15) - (-2.4980018054066e-16*z2 + 0.00778866513194084*sqrt(100.0*z3 + 981.0) + 1.66533453693773e-16)*(0.543859649122807*u_pwm**2 - 82.6725146198832*z2 + 4.50497212344554*sqrt(100.0*z3 + 981.0) + 55.5555555555557)]])


E/A Linearisierung

In [20]:
a0,a1,a2,a3=alpha=sp.symbols('alpha0:4')
u_input=sp.Symbol('u_{input}')
w=sp.Symbol('w') #Führungsgröße
print(alpha,u_input)

(alpha0, alpha1, alpha2, alpha3) u_{input}


In [21]:
u_input=1/(LgLf2h)*(a0*w-(a0*h+a1*Lfh+a2*Lf2h+a3*Lf3h))
u_input.subs([(w,0.8)])


1.83870967741935*(-alpha0*x1 + 0.8*alpha0 - alpha1*x2 - alpha2*(0.0778571428571429*(-6.57687834380088*x2 + 0.139567341242149*x3 + 4.41963247266806)**2 - 9.81) - alpha3*(-1.75438596491228*x3*(-0.142932856041132*x2 + 0.00303316522687554*x3 + 0.0960502321843922) + (0.0778571428571429*(-6.57687834380088*x2 + 0.139567341242149*x3 + 4.41963247266806)**2 - 9.81)*(6.7354726195116*x2 - 0.142932856041132*x3 - 4.52620710796917)))/(u_pwm*(-0.142932856041132*x2 + 0.00303316522687554*x3 + 0.0960502321843922))

In [22]:
s=sp.Symbol('s')
nenner=(s+10)*(s+5)*(s+7)
print(sp.expand(nenner))
#print(nenner.subs([(a0,10),(a1,2),(a2,3)]))
#sp.expand(sp.solve(nenner.subs([(a0,10),(a1,2),(a2,3)]),s)[0])

s**3 + 22*s**2 + 155*s + 350


In [23]:
sp.simplify(sp.expand(u_input.subs([(w,h_r),(a3,1),(a2,22),(a1,155),(a0,350)])))

(-643.548387096774*x1 - 41.7079953268847*x2**3 + 2.65524480400342*x2**2*x3 - 52.1476134356049*x2**2 - 0.0563467101365937*x2*x3**2 + 1.75216585924582*x2*x3 - 36.9181340001961*x2 + 0.000398576148663745*x3**3 - 0.0136990747338284*x3**2 - 4.9546834104035*x3 + 781.164489961175)/(u_pwm*(-0.142932856041132*x2 + 0.00303316522687554*x3 + 0.0960502321843922))

Zustand-Linearisierung

In [24]:
print(f_sys,g_sys)

Matrix([[x2], [0.0778571428571429*(-6.57687834380088*x2 + 0.139567341242149*x3 + 4.41963247266806)**2 - 9.81], [-1.75438596491228*x3]]) Matrix([[0], [0], [0.543859649122807*u_pwm]])


In [27]:
ad_fg=sp.simplify(st.lie_bracket(-f, g_sys, xx))
ad_f2g=sp.simplify(st.lie_bracket(-f, ad_fg , xx))
print(ad_fg,ad_f2g)

Matrix([[0], [u_pwm*(-0.0777354129346506*x2 + 0.00164961617602003*x3 + 0.0522378455739677)], [-0.95413973530317*u_pwm]]) Matrix([[u_pwm*(-0.0777354129346506*x2 + 0.00164961617602003*x3 + 0.0522378455739677)], [u_pwm*(-0.261792372693883*x2**2 + 0.0111109445862863*x2*x3 + 0.488224495994946*x2 - 0.000117892175705097*x3**2 - 0.00746650446132279*x3 - 0.972449397972091)], [1.673929360181*u_pwm]])


Distribution $\Delta_3$

In [39]:
Distribution_3=sp.Matrix([[g_sys[0],ad_fg[0],ad_f2g[0]],[g_sys[1],ad_fg[1],ad_f2g[1]],[g_sys[2],ad_fg[2],ad_f2g[2]]])
print(Distribution_3)

Matrix([[0, 0, u_pwm*(-0.0777354129346506*x2 + 0.00164961617602003*x3 + 0.0522378455739677)], [0, u_pwm*(-0.0777354129346506*x2 + 0.00164961617602003*x3 + 0.0522378455739677), u_pwm*(-0.261792372693883*x2**2 + 0.0111109445862863*x2*x3 + 0.488224495994946*x2 - 0.000117892175705097*x3**2 - 0.00746650446132279*x3 - 0.972449397972091)], [0.543859649122807*u_pwm, -0.95413973530317*u_pwm, 1.673929360181*u_pwm]])


In [40]:
Distribution_3.rank()

3

Prüfen ob Distribution $\Delta_2$ involutiv ist

In [41]:
Inv_1=sp.simplify(st.lie_bracket(g_sys, ad_fg, xx))
print(Inv_1)

Matrix([[0], [0.000897159674677561*u_pwm**2], [0]]) Matrix([[0.000897159674677561*u_pwm**2], [u_pwm**2*(0.00604279442412064*x2 - 0.000128233594626596*x3 - 0.00406073049650889)], [0]]) Matrix([[u_pwm**2*(0.00604279442412065*x2 - 0.000128233594626596*x3 - 0.00563469483804847)], [u_pwm**2*(0.020350538194501*x2**2 - 0.00086371386550899*x2*x3 - 0.0379523328009788*x2 + 9.16439696020007e-6*x3**2 + 0.00103035424476564*x3 - 0.0457272120419037)], [0]])


In [50]:
Inv=sp.Matrix([[g_sys[0],ad_fg[0],Inv_1[0]],[g_sys[1],ad_fg[1],Inv_1[1]],[g_sys[2],ad_fg[2],Inv_1[2]]])

In [52]:
Inv.rank()


2

Berechnung des Kovektorfeldes $\omega$, nämlich $\Delta_{n-1}^{\bot}$

In [57]:
Distribution_2=sp.Matrix([[g_sys[0],ad_fg[0]],[g_sys[1],ad_fg[1]],[g_sys[2],ad_fg[2]]])
anni = st.nullspaceMatrix(Distribution_2.T).T
anni

Matrix([[1, 0, 0]])

Suche eines Skalarfeldes $h$ mit $dh \in \operatorname{span}{\omega (x)}$